# 🏗️ Model Architecture: Building the SLM from Scratch

This notebook implements the complete model architecture for our 300M parameter SLM. We build everything from scratch:

- **Rotary Position Embeddings (RoPE)** - for relative positional encoding
- **Multi-Head Self-Attention** - the core of the transformer
- **Block-Local Sparse Attention** - for efficient long-context handling
- **Feed-Forward Networks** - the MLP layers

**Target Config:**
- Layers: 24
- d_model: 1024
- Heads: 16
- FFN: 4096
- ~300M parameters

---
## 1. Setup & Configuration

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
from typing import Optional, Tuple

# Check if we have a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# For reproducibility
torch.manual_seed(42)

In [ ]:
@dataclass
class ModelConfig:
    """All the knobs we can turn for our model."""
    
    # Vocabulary (from our tokenizer)
    vocab_size: int = 32000
    
    # Model dimensions
    d_model: int = 1024        # Hidden size
    n_layers: int = 24         # Number of transformer blocks
    n_heads: int = 16          # Attention heads
    d_ff: int = 4096           # Feed-forward intermediate size (4x d_model)
    
    # Context
    max_seq_len: int = 2048    # Start with 2k, extend later
    
    # Block-local attention (for long context)
    block_size: int = 512      # Each block attends locally
    use_block_local: bool = False  # Off for Phase A, on for Phase B
    
    # Training
    dropout: float = 0.1
    
    @property
    def head_dim(self):
        return self.d_model // self.n_heads

# Create our config
config = ModelConfig()
print(f"Head dimension: {config.head_dim}")
print(f"Max sequence length: {config.max_seq_len}")

---
## 2. Rotary Position Embeddings (RoPE)

RoPE encodes position by **rotating** the query and key vectors. This has nice properties:
- Position is encoded in the *relationship* between tokens, not absolute positions
- Can extrapolate to longer sequences than trained on
- No learned parameters - just math!

The key idea: for position `m`, we rotate each pair of dimensions by `m * θ`.

In [ ]:
class RotaryEmbedding(nn.Module):
    """
    Rotary Position Embeddings.
    
    The magic: we precompute sin/cos values for all positions,
    then apply them as rotations to Q and K vectors.
    """
    
    def __init__(self, dim: int, max_seq_len: int = 2048, base: float = 10000.0):
        super().__init__()
        self.dim = dim
        self.max_seq_len = max_seq_len
        self.base = base
        
        # Precompute the frequency bands
        # θ_i = base^(-2i/d) for i = 0, 1, ..., d/2-1
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer('inv_freq', inv_freq)
        
        # Precompute sin/cos for all positions
        self._build_cache(max_seq_len)
    
    def _build_cache(self, seq_len: int):
        """Precompute sin and cos values."""
        positions = torch.arange(seq_len).float()
        
        # Outer product: [seq_len] x [dim/2] -> [seq_len, dim/2]
        freqs = torch.outer(positions, self.inv_freq)
        
        # Stack to get [seq_len, dim] - each pair of dims shares a freq
        emb = torch.cat([freqs, freqs], dim=-1)
        
        self.register_buffer('cos_cached', emb.cos())
        self.register_buffer('sin_cached', emb.sin())
    
    def forward(self, x: torch.Tensor, seq_len: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """Return cos and sin for the given sequence length."""
        if seq_len > self.max_seq_len:
            # Extend cache if needed (this is how we handle longer contexts)
            self._build_cache(seq_len)
        
        return (
            self.cos_cached[:seq_len].to(x.device),
            self.sin_cached[:seq_len].to(x.device)
        )


def rotate_half(x: torch.Tensor) -> torch.Tensor:
    """Rotate half the hidden dims of x."""
    x1, x2 = x[..., :x.shape[-1]//2], x[..., x.shape[-1]//2:]
    return torch.cat([-x2, x1], dim=-1)


def apply_rotary_emb(q: torch.Tensor, k: torch.Tensor, 
                     cos: torch.Tensor, sin: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Apply rotary embeddings to query and key tensors.
    
    This is the actual rotation: x * cos + rotate_half(x) * sin
    """
    # Reshape for broadcasting: [seq_len, dim] -> [1, seq_len, 1, dim]
    cos = cos.unsqueeze(0).unsqueeze(2)
    sin = sin.unsqueeze(0).unsqueeze(2)
    
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    
    return q_embed, k_embed

# Quick test
rope = RotaryEmbedding(dim=64, max_seq_len=2048)
cos, sin = rope(torch.randn(1), seq_len=100)
print(f"RoPE cache shape: cos={cos.shape}, sin={sin.shape}")

---
## 3. Multi-Head Self-Attention

The core of the transformer. Each head learns different attention patterns.

In [ ]:
class MultiHeadAttention(nn.Module):
    """
    Multi-head self-attention with RoPE.
    
    For a decoder model, we use causal masking - each token can only
    attend to previous tokens (and itself).
    """
    
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.n_heads = config.n_heads
        self.head_dim = config.head_dim
        self.d_model = config.d_model
        
        # Q, K, V projections (could be fused, but this is clearer)
        self.q_proj = nn.Linear(config.d_model, config.d_model, bias=False)
        self.k_proj = nn.Linear(config.d_model, config.d_model, bias=False)
        self.v_proj = nn.Linear(config.d_model, config.d_model, bias=False)
        self.out_proj = nn.Linear(config.d_model, config.d_model, bias=False)
        
        # RoPE for this layer
        self.rotary = RotaryEmbedding(self.head_dim, config.max_seq_len)
        
        self.dropout = nn.Dropout(config.dropout)
        
        # Scaling factor for attention scores
        self.scale = 1.0 / math.sqrt(self.head_dim)
    
    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        batch_size, seq_len, _ = x.shape
        
        # Project to Q, K, V
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)
        
        # Reshape: [batch, seq, d_model] -> [batch, seq, n_heads, head_dim]
        q = q.view(batch_size, seq_len, self.n_heads, self.head_dim)
        k = k.view(batch_size, seq_len, self.n_heads, self.head_dim)
        v = v.view(batch_size, seq_len, self.n_heads, self.head_dim)
        
        # Apply RoPE to Q and K (not V!)
        cos, sin = self.rotary(q, seq_len)
        q, k = apply_rotary_emb(q, k, cos, sin)
        
        # Transpose for attention: [batch, n_heads, seq, head_dim]
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)
        
        # Attention scores: [batch, n_heads, seq, seq]
        attn = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        
        # Causal mask - each position can only see previous positions
        if mask is None:
            mask = torch.triu(torch.ones(seq_len, seq_len, device=x.device), diagonal=1).bool()
        attn = attn.masked_fill(mask, float('-inf'))
        
        # Softmax and dropout
        attn = F.softmax(attn, dim=-1)
        attn = self.dropout(attn)
        
        # Apply attention to values
        out = torch.matmul(attn, v)  # [batch, n_heads, seq, head_dim]
        
        # Reshape back: [batch, seq, d_model]
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        
        return self.out_proj(out)

# Test it
attn = MultiHeadAttention(config)
x = torch.randn(2, 128, config.d_model)  # [batch=2, seq=128, d_model]
out = attn(x)
print(f"Attention output shape: {out.shape}")

---
## 4. Block-Local Sparse Attention

For long contexts (4k-5k tokens), full attention becomes expensive: O(N²).

**Block-local attention** splits the sequence into blocks. Each token attends to:
- Its own block (full attention within)
- The previous block (for continuity)

This gives us O(N × block_size) instead of O(N²).

In [ ]:
class BlockLocalAttention(nn.Module):
    """
    Block-local sparse attention for efficient long-context modeling.
    
    Each block of 'block_size' tokens attends to itself and the previous block.
    This is much more efficient for long sequences.
    """
    
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.n_heads = config.n_heads
        self.head_dim = config.head_dim
        self.d_model = config.d_model
        self.block_size = config.block_size
        
        self.q_proj = nn.Linear(config.d_model, config.d_model, bias=False)
        self.k_proj = nn.Linear(config.d_model, config.d_model, bias=False)
        self.v_proj = nn.Linear(config.d_model, config.d_model, bias=False)
        self.out_proj = nn.Linear(config.d_model, config.d_model, bias=False)
        
        self.rotary = RotaryEmbedding(self.head_dim, config.max_seq_len)
        self.dropout = nn.Dropout(config.dropout)
        self.scale = 1.0 / math.sqrt(self.head_dim)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        batch_size, seq_len, _ = x.shape
        block_size = self.block_size
        
        # Pad sequence to be divisible by block_size
        pad_len = (block_size - seq_len % block_size) % block_size
        if pad_len > 0:
            x = F.pad(x, (0, 0, 0, pad_len))
        
        padded_len = x.shape[1]
        n_blocks = padded_len // block_size
        
        # Project Q, K, V
        q = self.q_proj(x).view(batch_size, padded_len, self.n_heads, self.head_dim)
        k = self.k_proj(x).view(batch_size, padded_len, self.n_heads, self.head_dim)
        v = self.v_proj(x).view(batch_size, padded_len, self.n_heads, self.head_dim)
        
        # Apply RoPE
        cos, sin = self.rotary(q, padded_len)
        q, k = apply_rotary_emb(q, k, cos, sin)
        
        # Reshape into blocks: [batch, n_blocks, block_size, n_heads, head_dim]
        q = q.view(batch_size, n_blocks, block_size, self.n_heads, self.head_dim)
        k = k.view(batch_size, n_blocks, block_size, self.n_heads, self.head_dim)
        v = v.view(batch_size, n_blocks, block_size, self.n_heads, self.head_dim)
        
        # For each block, we attend to current + previous block
        # Shift K and V to create "previous block" context
        k_prev = F.pad(k, (0, 0, 0, 0, 0, 0, 1, 0))[:, :-1]  # Shift right, pad start
        v_prev = F.pad(v, (0, 0, 0, 0, 0, 0, 1, 0))[:, :-1]
        
        # Concatenate current and previous block keys/values
        # [batch, n_blocks, 2*block_size, n_heads, head_dim]
        k_local = torch.cat([k_prev, k], dim=2)
        v_local = torch.cat([v_prev, v], dim=2)
        
        # Transpose for attention: [batch, n_blocks, n_heads, block_size, 2*block_size]
        q = q.transpose(2, 3)
        k_local = k_local.transpose(2, 3)
        v_local = v_local.transpose(2, 3)
        
        # Compute attention
        attn = torch.matmul(q, k_local.transpose(-2, -1)) * self.scale
        
        # Causal mask within the local window
        # Query position i in block can attend to key positions 0..(block_size + i)
        mask = torch.ones(block_size, 2 * block_size, device=x.device).bool()
        for i in range(block_size):
            # Can attend to previous block + positions 0..i in current block
            mask[i, :block_size + i + 1] = False
        
        attn = attn.masked_fill(mask, float('-inf'))
        attn = F.softmax(attn, dim=-1)
        attn = self.dropout(attn)
        
        # Apply attention
        out = torch.matmul(attn, v_local)
        
        # Reshape back
        out = out.transpose(2, 3).contiguous()
        out = out.view(batch_size, padded_len, self.d_model)
        
        # Remove padding
        if pad_len > 0:
            out = out[:, :seq_len]
        
        return self.out_proj(out)

# Test it
block_attn = BlockLocalAttention(config)
x = torch.randn(2, 1024, config.d_model)
out = block_attn(x)
print(f"Block-local attention output shape: {out.shape}")

---
## 5. Feed-Forward Network

The other half of each transformer block - a simple 2-layer MLP with GeLU activation.

In [ ]:
class FeedForward(nn.Module):
    """
    Position-wise feed-forward network.
    
    Two linear layers with GELU activation in between.
    Expands from d_model to d_ff (4x), then back down.
    """
    
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.fc1 = nn.Linear(config.d_model, config.d_ff, bias=False)
        self.fc2 = nn.Linear(config.d_ff, config.d_model, bias=False)
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Expand -> activate -> contract
        x = self.fc1(x)
        x = F.gelu(x)  # Smoother than ReLU
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Test
ffn = FeedForward(config)
out = ffn(torch.randn(2, 128, config.d_model))
print(f"FFN output shape: {out.shape}")

---
## 6. Transformer Block

Now we combine attention + feed-forward with residual connections and layer norm.

In [ ]:
class TransformerBlock(nn.Module):
    """
    A single transformer block: attention + FFN with residuals.
    
    We use pre-norm (LayerNorm before each sublayer) - this tends to
    train more stably than post-norm.
    """
    
    def __init__(self, config: ModelConfig):
        super().__init__()
        
        # Pre-norm layers
        self.attn_norm = nn.LayerNorm(config.d_model)
        self.ffn_norm = nn.LayerNorm(config.d_model)
        
        # Choose attention type based on config
        if config.use_block_local:
            self.attention = BlockLocalAttention(config)
        else:
            self.attention = MultiHeadAttention(config)
        
        self.ffn = FeedForward(config)
        self.dropout = nn.Dropout(config.dropout)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Attention with residual
        h = self.attn_norm(x)
        h = self.attention(h)
        x = x + self.dropout(h)
        
        # FFN with residual
        h = self.ffn_norm(x)
        h = self.ffn(h)
        x = x + self.dropout(h)
        
        return x

# Test
block = TransformerBlock(config)
out = block(torch.randn(2, 128, config.d_model))
print(f"Transformer block output shape: {out.shape}")

---
## 7. Complete Model

The full decoder model: embeddings -> N transformer blocks -> output projection.

In [ ]:
class SLM(nn.Module):
    """
    The complete Small Language Model.
    
    Decoder-only transformer for autoregressive language modeling.
    """
    
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.config = config
        
        # Token embeddings
        self.token_emb = nn.Embedding(config.vocab_size, config.d_model)
        
        # Stack of transformer blocks
        self.blocks = nn.ModuleList([
            TransformerBlock(config) for _ in range(config.n_layers)
        ])
        
        # Final layer norm (before output projection)
        self.final_norm = nn.LayerNorm(config.d_model)
        
        # Output projection (ties with token embeddings for efficiency)
        self.output = nn.Linear(config.d_model, config.vocab_size, bias=False)
        
        # Weight tying - embedding and output share weights
        self.output.weight = self.token_emb.weight
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
        """Initialize weights with small values for stable training."""
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.normal_(p, mean=0.0, std=0.02)
    
    def forward(self, tokens: torch.Tensor) -> torch.Tensor:
        """
        Forward pass.
        
        Args:
            tokens: [batch, seq_len] token IDs
        
        Returns:
            logits: [batch, seq_len, vocab_size] predictions for each position
        """
        # Embed tokens
        x = self.token_emb(tokens)  # [batch, seq, d_model]
        
        # Pass through transformer blocks
        for block in self.blocks:
            x = block(x)
        
        # Final norm and project to vocabulary
        x = self.final_norm(x)
        logits = self.output(x)
        
        return logits
    
    def generate(self, prompt_tokens: torch.Tensor, max_new_tokens: int = 100,
                 temperature: float = 1.0) -> torch.Tensor:
        """
        Generate text autoregressively.
        
        Simple greedy/temperature sampling - nothing fancy.
        """
        self.eval()
        tokens = prompt_tokens.clone()
        
        with torch.no_grad():
            for _ in range(max_new_tokens):
                # Get predictions for last position
                logits = self(tokens)[:, -1, :]  # [batch, vocab]
                
                # Apply temperature
                logits = logits / temperature
                
                # Sample next token
                probs = F.softmax(logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
                
                # Append to sequence
                tokens = torch.cat([tokens, next_token], dim=1)
                
                # Stop at max length
                if tokens.shape[1] >= self.config.max_seq_len:
                    break
        
        return tokens

---
## 8. Parameter Count

Let's verify we're in the ~300M range.

In [ ]:
def count_parameters(model):
    """Count trainable parameters."""
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

# Create the model
model = SLM(config)

total, trainable = count_parameters(model)
print(f"Total parameters: {total:,}")
print(f"Trainable parameters: {trainable:,}")
print(f"\nThat's approximately {total / 1e6:.0f}M parameters")

# Breakdown by component
print("\nParameter breakdown:")
print(f"  Token embeddings: {model.token_emb.weight.numel():,}")
print(f"  Transformer blocks: {sum(p.numel() for p in model.blocks.parameters()):,}")
print(f"  Final norm: {sum(p.numel() for p in model.final_norm.parameters()):,}")

In [ ]:
# Quick forward pass test
batch_size = 2
seq_len = 256

# Random token IDs
dummy_tokens = torch.randint(0, config.vocab_size, (batch_size, seq_len))

# Forward pass
logits = model(dummy_tokens)
print(f"Input shape: {dummy_tokens.shape}")
print(f"Output shape: {logits.shape}")
print(f"\n✓ Model works!")

---
## Summary

We've built:

1. **RoPE** - Rotary position embeddings for relative position encoding
2. **Multi-Head Attention** - Standard causal self-attention with RoPE
3. **Block-Local Attention** - Efficient sparse attention for long contexts
4. **Transformer Block** - Attention + FFN with pre-norm and residuals
5. **Complete SLM** - ~300M parameter decoder model

**Next:** In notebook 04, we'll set up the training pipeline to actually train this model.

In [ ]:
# Save config for later use
import json
from pathlib import Path

config_path = Path("../configs")
config_path.mkdir(exist_ok=True)

# Save as JSON
config_dict = {
    'vocab_size': config.vocab_size,
    'd_model': config.d_model,
    'n_layers': config.n_layers,
    'n_heads': config.n_heads,
    'd_ff': config.d_ff,
    'max_seq_len': config.max_seq_len,
    'block_size': config.block_size,
    'dropout': config.dropout
}

with open(config_path / 'model_config.json', 'w') as f:
    json.dump(config_dict, f, indent=2)

print(f"✓ Config saved to {config_path / 'model_config.json'}")